In [ ]:
# Agent's decisions in the No governance case, at https://ivanjureta.com/how-can-governance-of-attention-and-memory-change-choice/

import numpy as np
import random
from datetime import datetime
import drawsvg as dw
import matplotlib.pyplot as plt
import cairosvg
import os

def ij_agent_simulation_no_governance(grid_rows, grid_columns, run_count, image_width, image_height,
                                      grid_edge_thickness, grid_edge_color, grid_center_color, 
                                      path_thickness, step_number_color, path_cell_color, 
                                      image_background_color, make_images):
    # Results to calculate average and standard deviation
    all_steps = []

    # Define possible moves for the agent
    def get_adjacent_cells(x, y):
        moves = []
        if x > 1: moves.append((x - 1, y))  # Move up
        if x < grid_rows: moves.append((x + 1, y))  # Move down
        if y > 1: moves.append((x, y - 1))  # Move left
        if y < grid_columns: moves.append((x, y + 1))  # Move right
        return moves

    for run in range(run_count):
        x, y = 1, 1  # Starting position
        steps = 0
        visited_cells = {(x, y): 1}  # Count of visits per cell
        path = [(x, y)]  # Track the path

        # Run until the agent reaches the target cell
        while (x, y) != (grid_rows, grid_columns):
            adjacent_cells = get_adjacent_cells(x, y)
            x, y = random.choice(adjacent_cells)
            steps += 1
            path.append((x, y))
            if (x, y) in visited_cells:
                visited_cells[(x, y)] += 1
            else:
                visited_cells[(x, y)] = 1

        all_steps.append(steps)

        if make_images == 1:
            # Create unique filename
            date_str = datetime.now().strftime("%Y%m%d")
            random_number = random.randint(100000, 999999)
            filename = f"ivanjureta_com_agent_walk_{date_str}_{random_number:06}_{run + 1}"

            # SVG creation
            drawing = dw.Drawing(image_width, image_height, origin="center")
            drawing.append(dw.Rectangle(-image_width / 2, -image_height / 2, image_width, image_height, fill=image_background_color))

            # Create grid
            cell_width = image_width / grid_columns
            cell_height = image_height / grid_rows

            for i in range(grid_rows):
                for j in range(grid_columns):
                    x_pos = -image_width / 2 + j * cell_width
                    y_pos = -image_height / 2 + i * cell_height
                    # Draw cell border
                    drawing.append(dw.Rectangle(x_pos, y_pos, cell_width, cell_height, stroke=grid_edge_color, stroke_width=grid_edge_thickness, fill='none'))
                    # Draw center square
                    center_square_size = path_thickness
                    center_x = x_pos + (cell_width - center_square_size) / 2
                    center_y = y_pos + (cell_height - center_square_size) / 2
                    drawing.append(dw.Rectangle(center_x, center_y, center_square_size, center_square_size, fill=grid_center_color))

            # Draw agent path with transparency in path color
            for cell, visits in visited_cells.items():
                i, j = cell
                x_pos = -image_width / 2 + (j - 1) * cell_width
                y_pos = -image_height / 2 + (i - 1) * cell_height
                alpha = min(1.0, visits * 0.1)
                drawing.append(dw.Circle(x_pos + cell_width / 2, y_pos + cell_height / 2, cell_width / 2, fill=path_cell_color, fill_opacity=alpha))
                # Add visit count
                drawing.append(dw.Text(str(visits), font_size=cell_width / 3, x=x_pos + cell_width / 2, y=y_pos + cell_height / 2, fill=step_number_color, font_family="Noto Sans", text_anchor="middle", dominant_baseline="middle"))

            # Save SVG
            svg_filepath = f"{filename}.svg"
            drawing.save_svg(svg_filepath)

            # Convert SVG to PNG using cairosvg
            png_filepath = f"{filename}.png"
            with open(svg_filepath, "rb") as svg_file:
                cairosvg.svg2png(file_obj=svg_file, write_to=png_filepath)

            # Save text file with details
            text_filepath = f"{filename}.txt"
            with open(text_filepath, "w") as text_file:
                text_file.write(f"Python Script: \n{ij_agent_simulation_no_governance.__code__}\n\n")
                text_file.write(f"Variables:\n")
                text_file.write(f"Grid Rows: {grid_rows}\nGrid Columns: {grid_columns}\nRun Count: {run_count}\n")
                text_file.write(f"Image Width: {image_width}\nImage Height: {image_height}\n")
                text_file.write(f"Grid Edge Thickness: {grid_edge_thickness}\nGrid Edge Color: {grid_edge_color}\n")
                text_file.write(f"Grid Center Color: {grid_center_color}\nPath Thickness: {path_thickness}\n")
                text_file.write(f"Step Number Color: {step_number_color}\nPath Cell Color: {path_cell_color}\n")
                text_file.write(f"Image Background Color: {image_background_color}\n")
                text_file.write(f"Run: {run + 1}\nSteps Taken: {steps}\nPath:\n{path}\nTotal Steps: {steps}\n")

    # Calculate average and standard deviation of steps
    avg_steps = np.mean(all_steps)
    std_dev_steps = np.std(all_steps)
    print(f"Average number of steps: {avg_steps}")
    print(f"Standard deviation of steps: {std_dev_steps}")

    # Plot histogram of steps
    plt.hist(all_steps, bins=20, edgecolor='black')
    plt.title("Histogram of Number of Steps Across Runs")
    plt.xlabel("Number of Steps")
    plt.ylabel("Frequency")
    plt.show()


In [ ]:
# Simulate agent's decisions
# Be careful when setting make_images: if 1, then the number of output files will be 3 * run_count 
ij_agent_simulation_no_governance(
    grid_rows=20, 
    grid_columns=40, 
    run_count=1000, 
    image_width=2000, 
    image_height=1000, 
    image_background_color='none', 
    make_images = 0,
    grid_edge_thickness=0, 
    grid_edge_color='none', 
    path_thickness=3, 
    grid_center_color='#cccccc', 
    path_cell_color='#FF003C',  
    step_number_color='#ffffff'
)